In [9]:
import pandas as pd
import numpy as np
import pickle as pkl

df= pd.read_pickle('TXF_1m.pkl')
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.columns = ['open', 'high', 'low', 'close', 'volume', 'date', 'time']

df



,open,high,low,close,volume,date,time
datetime,,,,,,,
2010-01-04 08:46:00,8203.0,8208.0,8184.0,8187.0,1108,2010-01-04,08:46:00
2010-01-04 08:47:00,8187.0,8195.0,8186.0,8193.0,429,2010-01-04,08:47:00
2010-01-04 08:48:00,8194.0,8201.0,8193.0,8200.0,419,2010-01-04,08:48:00
2010-01-04 08:49:00,8200.0,8203.0,8198.0,8200.0,295,2010-01-04,08:49:00
2010-01-04 08:50:00,8200.0,8201.0,8195.0,8197.0,187,2010-01-04,08:50:00
...,...,...,...,...,...,...,...
2024-07-12 15:53:00,23833.0,23837.0,23833.0,23834.0,22,2024-07-12,15:53:00
2024-07-12 15:54:00,23832.0,23832.0,23828.0,23831.0,53,2024-07-12,15:54:00
2024-07-12 15:55:00,23833.0,23840.0,23833.0,23838.0,68,2024-07-12,15:55:00


## ATR 


In [15]:
# Calculate True Range (TR)
df['previous_close'] = df['close'].shift(1)
df['high_low'] = df['high'] - df['low']
df['high_prev_close'] = (df['high'] - df['previous_close']).abs()
df['low_prev_close'] = (df['low'] - df['previous_close']).abs()

df['TR'] = df[['high_low', 'high_prev_close', 'low_prev_close']].max(axis=1)

# Calculate ATR
df['ATR'] = df['TR'].rolling(window=5).mean()  # You can adjust the window size

# Calculate velocity (price change with direction)
df['velocity'] = df['close'].diff()

# Incorporate ATR into the velocity component (additive approach)
df['adj_velocity'] = 0.4*df['velocity'] +0.6*df['ATR']

# Adjust mass by incorporating ATR
df['adj_mass'] = 241000 * df['ATR'] 

# Calculate adjusted kinetic energy using both ATR in mass and velocity
df['adj_kinetic_energy'] = 0.5 * df['adj_mass'] * df['adj_velocity']**2

# Calculate adjusted momentum using both ATR in mass and velocity
df['adj_momentum'] = df['adj_mass'] * df['adj_velocity']
df['adj_momentum'].mean()
# Display the results
df

,open,high,low,close,volume,date,time,previous_close,high_low,high_prev_close,...,ATR,velocity,adjusted_velocity,adjusted_mass,adjusted_kinetic_energy,adjusted_momentum,adj_velocity,adj_mass,adj_kinetic_energy,adj_momentum
datetime,,,,,,,,,,,,,,,,,,,,,
2010-01-04 08:46:00,8203.0,8208.0,8184.0,8187.0,1108,2010-01-04,08:46:00,NaN,24.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04 08:47:00,8187.0,8195.0,8186.0,8193.0,429,2010-01-04,08:47:00,8187.0,9.0,8.0,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04 08:48:00,8194.0,8201.0,8193.0,8200.0,419,2010-01-04,08:48:00,8193.0,8.0,8.0,...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04 08:49:00,8200.0,8203.0,8198.0,8200.0,295,2010-01-04,08:49:00,8200.0,5.0,3.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04 08:50:00,8200.0,8201.0,8195.0,8197.0,187,2010-01-04,08:50:00,8200.0,6.0,1.0,...,10.4,-3.0,5.04,2506400.0,31833285.12,12632256.0,5.04,2506400.0,31833285.12,12632256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-12 15:53:00,23833.0,23837.0,23833.0,23834.0,22,2024-07-12,15:53:00,23833.0,4.0,4.0,...,6.2,1.0,4.12,1494200.0,12681574.24,6156104.0,4.12,1494200.0,12681574.24,6156104.0
2024-07-12 15:54:00,23832.0,23832.0,23828.0,23831.0,53,2024-07-12,15:54:00,23834.0,4.0,2.0,...,5.8,-3.0,2.28,1397800.0,3633161.76,3186984.0,2.28,1397800.0,3633161.76,3186984.0
2024-07-12 15:55:00,23833.0,23840.0,23833.0,23838.0,68,2024-07-12,15:55:00,23831.0,7.0,9.0,...,6.0,7.0,6.40,1446000.0,29614080.00,9254400.0,6.40,1446000.0,29614080.00,9254400.0
